# Best Models
Chan Wei Jian Ivan
1005924
Tom Manuel Opalla Piccio
1006293
Deshpande Sunny Nitin
1006336



In [47]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import pytz
import numpy as np
import os
import sys
import glob
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import matplotlib.pyplot as plt
import re
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import ConcatDataset, TensorDataset

In [6]:
# Define the directory and base filename pattern
file_path_test = "data/test_data/cleaned_compiled_data_normalized.csv"

# Load and concatenate all parts
test_data = pd.read_csv(file_path_test)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.set_default_device(device)

g = torch.Generator(device=device)

cuda


In [29]:
def safe_sigmoid(x):
    # If all outputs are already in [0, 1], assume sigmoid already applied
    if x.min() >= 0 and x.max() <= 1:
        return x  # Already probabilities
    return torch.sigmoid(x)  # Assume logits

def evaluate_metrics(dataloader, model, device="cuda"):
    model.eval()
    model.to(device)
    all_preds, all_targets = [], []

    with torch.no_grad():
        for inputs, targets in dataloader:
            # Convert from NumPy if needed
            if isinstance(inputs, np.ndarray):
                inputs = torch.tensor(inputs)
            if isinstance(targets, np.ndarray):
                targets = torch.tensor(targets)

            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            
            probs = safe_sigmoid(outputs)
            preds = (probs > 0.5).int()

            all_preds.append(preds.cpu())
            all_targets.append(targets.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()

    accuracy = (all_preds == all_targets).mean() * 100
    precision = precision_score(all_targets, all_preds, average="macro", zero_division=0)
    recall = recall_score(all_targets, all_preds, average="macro", zero_division=0)
    f1 = f1_score(all_targets, all_preds, average="macro", zero_division=0)

    print("🔎 Evaluation Results for LSTM Model")
    print("====================================")
    print(f"✅ Accuracy : {accuracy:.2f}%")
    print(f"🎯 Precision: {precision:.4f}")
    print(f"🔁 Recall   : {recall:.4f}")
    print(f"📊 F1 Score : {f1:.4f}")

    return accuracy, precision, recall, f1


In [33]:
from LSTM import LSTM_Dataset,LSTM_Module
LSTM_dataset = LSTM_Dataset(test_data)
LSTM_dataloader = DataLoader(LSTM_dataset, batch_size=64, shuffle=True, generator=g)

d:\Documents\Term 8\Deep Learning\Lightning_Risk_Prediction\LSTM\LSTM.py:48: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  self.compiled_df[col] = self.compiled_df[col].interpolate(limit_direction='both')


In [31]:
LSTM_model = LSTM_Module(
    hours_lookback=6,  # adjust if different
    input_dim=35,      # make sure it matches your input feature size
    hidden_dim=128,
    num_layers=1,
    dropout=0.2
)
LSTM_model_path = r'LSTM/LSTM.pth'
LSTM_model.load_state_dict(torch.load(LSTM_model_path, map_location=device))
LSTM_model.to(device)
LSTM_model.eval()

C:\Users\tomma\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


LSTM_Module(
  (lstm): LSTM(35, 128, batch_first=True, dropout=0.2)
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=5, bias=True)
  )
)

In [46]:
LSTM_results = evaluate_metrics(LSTM_dataloader,LSTM_model)

🔎 Evaluation Results for LSTM Model
✅ Accuracy : 93.21%
🎯 Precision: 0.4831
🔁 Recall   : 0.1989
📊 F1 Score : 0.2627


In [44]:
chosen_stations = ['S104', 'S107', 'S109', 'S115', 'S116', 'S43', 'S50']
pixel_coords = [(4, 11, 'S109'),
 (2, 7, 'S50'),
 (1, 16, 'S107'),
 (2, 13, 'S43'),
 (0, 0, 'S115'),
 (0, 6, 'S116'),
 (8, 8, 'S104')]


In [50]:
from CNN_RNN import CNN_RNN_Dataset, CNN_RNN_Module


In [51]:
CNN_RNN_dataset = CNN_RNN_Dataset(test_data,pixel_coords)
CNN_dataloader = DataLoader(LSTM_Dataset, batch_size=64, shuffle=True, generator=g)

NameError: name 'tabular_to_image' is not defined